In [1]:
!pip install transformers

In [2]:
!pip install torch

In [3]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gbptr12x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gbptr12x
  Resolved https://github.com/huggingface/transformers to commit 78b2929c0554b79e0489b451ce4ece14d265ead2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9786808 sha256=cc499411352d2491acedb46934f898f439ac0fcc9575b62c245062dd66eed543
  Stored in directory: /tmp/pip-ephem-wheel-cache-p1j7s2rk/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [4]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-sg7pzuvr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-sg7pzuvr
  Resolved https://github.com/huggingface/accelerate to commit 4305033f8035defad0a87cd38e5c918e78510ba5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.35.0.dev0-py3-none-any.whl size=330627 sha256=a2e26c03d1d8e2d0c95c9f41334e4e18209d9ad86d56fd1d89dfc5682874cf20
  Stored in directory: /tmp/pip-ephem-wheel-cache-v8ftl3ue/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [5]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 40.5 MB/s eta 0:00:0000:01:00:01


In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import os
import sys
import pandas as pd
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import traceback

def process_folder(folder_path):
    # Load the model and processor
    model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="cpu")
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

    results = []
    messages = []

    # Check if the folder exists and list its contents
    if not os.path.exists(folder_path):
        raise ValueError(f"The folder {folder_path} does not exist.")
    
    files = os.listdir(folder_path)
    print(f"Files in {folder_path}: {files}")  # Print out files in the directory

    if not files:
        raise ValueError(f"The folder {folder_path} is empty.")

    # Loop through images in the specified folder
    for image_name in files:
        if image_name.endswith(('.png', '.jpg', '.jpeg')):  # Add more formats if needed
            image_path = os.path.join(folder_path, image_name)

            # Try to load the image
            try:
                image = Image.open(image_path)
                print(f"Successfully loaded: {image_name}")

                # Add the image to the messages
                messages.append(
                    {
                        "role": "user",
                        "content": [
                            {"type": "image", "image": image},
                            {"type": "text", "text": "Describe this image."},
                        ],
                    }
                )
            except Exception as e:
                print(f"Failed to load {image_name}: {e}")
                print(traceback.format_exc())

    # Check if messages list is populated
    print(f"Number of messages: {len(messages)}")
    if len(messages) == 0:
        raise ValueError("No messages to process. Ensure that images are loaded correctly.")

    # Preparation for batch inference
    texts = [
        processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
        for msg in messages
    ]

    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=texts,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    # Ensure inputs are on the CPU
    inputs = inputs.to("cpu")
    model = model.to("cpu")

    # Generate output for all images simultaneously
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    # Decode the output texts
    output_texts = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # Store results in a list for DataFrame creation
    for i, output_text in enumerate(output_texts):
        results.append({"Image Name": os.listdir(folder_path)[i], "Description": output_text})

    # Convert results to a DataFrame and save as Excel
    df = pd.DataFrame(results, columns=["Image Name", "Description"])
    df.to_excel("diagnosis_results.xlsx", index=False)

    print("Results saved to diagnosis_results.xlsx")

def main():
    folder_path = "/kaggle/input/medical-diagonisis"  # Set your image path here correctly
    process_folder(folder_path)

if __name__ == "__main__":
    main()

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Files in /kaggle/input/medical-diagonisis: ['Sample17.png', 'Sample24.png', 'Sample29.png', 'Sample23.png', 'Sample14.png', 'Sample5.png', 'Sample11.png', 'Sample4.png', 'Sample8.png', 'Sample19.png', 'Sample9.png', 'Sample18.png', 'Sample22.png', 'Sample12.png', 'Sample15.png', 'Sample27.png', 'Sample20.png', 'Sample1.png', 'Sample26.png', 'Sample13.png', 'Sample6.png', 'Sample30.png', 'Sample10.png', 'Sample25.png', 'Sample7.png', 'Sample16.png', 'Sample28.png', 'Sample3.png', 'Sample21.png', 'Sample2.png']
Successfully loaded: Sample17.png
Successfully loaded: Sample24.png
Successfully loaded: Sample29.png
Successfully loaded: Sample23.png
Successfully loaded: Sample14.png
Successfully loaded: Sample5.png
Successfully loaded: Sample11.png
Successfully loaded: Sample4.png
Successfully loaded: Sample8.png
Successfully loaded: Sample19.png
Successfully loaded: Sample9.png
Successfully loaded: Sample18.png
Successfully loaded: Sample22.png
Successfully loaded: Sample12.png
Successfully 